# Experiment 10c: Restoration Forcing Sweep

**Phase 4 - Quantifying Active Intervention Requirements**

## Background

Experiments 10 and 10b demonstrated that **passive recovery is impossible** under any tested noise regime:
- Varying α (1.1-2.0): ~1.3% recovery
- Varying σ (0.04-0.12): ~0-4% recovery
- Best case (α=1.2, σ=0.10): only 3.6% recovery

**Key insight**: The barrier height creates insurmountable hysteresis - noise alone cannot push cells from the tipped attractor back to the stable state.

## This Experiment

Add **restoration forcing** during recovery phase - a constant negative term in dx/dt that pushes cells toward the stable state (x < 0). This models:
- Reforestation programs
- Fire suppression efforts
- Moisture corridor restoration
- Active ecosystem management

The forcing term adds `f` to dx/dt during recovery:
```
dx/dt = cusp_dynamics + coupling + f
```
where `f < 0` pushes toward stable state.

## Experimental Design

| Parameter | Values |
|-----------|--------|
| Forcing strength | [0.0, -0.05, -0.10, -0.15, -0.20, -0.30, -0.40, -0.50] |
| Recovery α | 2.0 (Gaussian - conservative) |
| Recovery σ | 0.04 (baseline from Exp 9) |
| Ensemble runs | 20 per forcing level |
| **Total simulations** | 160 |

## Research Questions

1. What forcing strength is needed for >50% recovery?
2. Is the forcing-recovery relationship linear or threshold-like?
3. How does forcing interact with noise?

## 1. Setup and Imports

In [ ]:
import sys
sys.path.insert(0, '/opt/research-local/src')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import time
from pathlib import Path
from netCDF4 import Dataset
from dask.distributed import as_completed

# Core energy-constrained module
from energy_constrained import (
    EnergyConstrainedNetwork,
    EnergyConstrainedCusp,
    GradientDrivenCoupling,
    run_two_phase_experiment,
    get_dask_client
)

print("Imports successful!")
print(f"NumPy version: {np.__version__}")

In [ ]:
# Connect to Dask cluster
client = get_dask_client()
print(f"Connected to: {client.scheduler_info()['address']}")
print(f"Workers: {len(client.scheduler_info()['workers'])}")
print(f"Total threads: {sum(w['nthreads'] for w in client.scheduler_info()['workers'].values())}")

## 2. Load Amazon Data

In [ ]:
DATA_PATH = Path('/opt/research-local/data/amazon/amazon_adaptation_model/average_network/era5_new_network_data')

def load_amazon_data(year=2003, months=[7, 8, 9]):
    """Load and average Amazon moisture recycling data."""
    all_rain = []
    all_evap = []
    all_network = []
    
    for month in months:
        file_path = DATA_PATH / f'1deg_{year}_{month:02d}.nc'
        if file_path.exists():
            with Dataset(file_path, 'r') as ds:
                all_rain.append(ds.variables['rain'][:])
                all_evap.append(ds.variables['evap'][:])
                all_network.append(ds.variables['network'][:])
    
    return {
        'rain': np.mean(all_rain, axis=0),
        'evap': np.mean(all_evap, axis=0),
        'network': np.mean(all_network, axis=0),
        'n_cells': len(all_rain[0])
    }

amazon_data = load_amazon_data(year=2003)
print(f"Loaded Amazon data: {amazon_data['n_cells']} cells")

## 3. Experiment Configuration

In [ ]:
# Experiment 10c: Restoration Forcing Sweep

SWEEP_CONFIG = {
    # Forcing sweep - negative values push toward stable state
    'forcing_values': np.array([0.0, -0.05, -0.10, -0.15, -0.20, -0.30, -0.40, -0.50]),
    'n_runs_per_forcing': 20,
    
    # Network parameters
    'n_cells': 50,
    'min_flow': 1.0,
    'barrier_height': 0.2,
    
    # Two-phase simulation parameters
    'cascade_duration': 200,
    'recovery_duration': 800,
    'dt': 0.5,
    'cascade_sigma': 0.06,
    'cascade_alpha': 1.5,
    'recovery_sigma': 0.04,  # Baseline from Exp 9
    'recovery_alpha': 2.0,   # Gaussian (conservative)
    
    # Seeds
    'base_seed': 42,
}

n_forcing = len(SWEEP_CONFIG['forcing_values'])
total_sims = n_forcing * SWEEP_CONFIG['n_runs_per_forcing']

print("=" * 60)
print("EXPERIMENT 10c: RESTORATION FORCING SWEEP")
print("=" * 60)
print(f"Forcing values: {SWEEP_CONFIG['forcing_values']}")
print(f"Runs per forcing level: {SWEEP_CONFIG['n_runs_per_forcing']}")
print(f"Total simulations: {total_sims}")
print(f"\nRecovery parameters (fixed):")
print(f"  α = {SWEEP_CONFIG['recovery_alpha']} (Gaussian)")
print(f"  σ = {SWEEP_CONFIG['recovery_sigma']}")
print(f"\nPhysical interpretation:")
print(f"  f=0.0:  No intervention (passive recovery)")
print(f"  f=-0.1: Moderate intervention")
print(f"  f=-0.3: Strong intervention")
print(f"  f=-0.5: Maximum intervention")

## 4. Network Creation

In [ ]:
def create_sweep_network(data, config, seed=42):
    """Create 50-cell network for forcing sweep."""
    np.random.seed(seed)
    
    network_matrix = data['network']
    n_cells = config['n_cells']
    min_flow = config['min_flow']
    barrier_height = config['barrier_height']
    
    total_flow = network_matrix.sum(axis=0) + network_matrix.sum(axis=1)
    top_indices = np.argsort(total_flow)[-n_cells:]
    
    net = EnergyConstrainedNetwork()
    
    for i, idx in enumerate(top_indices):
        element = EnergyConstrainedCusp(
            a=-1.0, b=1.0, c=0.0, x_0=0.0,
            barrier_height=barrier_height,
            dissipation_rate=0.1
        )
        net.add_element(f'cell_{i}', element)
    
    n_edges = 0
    for i, idx_i in enumerate(top_indices):
        for j, idx_j in enumerate(top_indices):
            if i != j:
                flow = network_matrix[idx_i, idx_j]
                if flow > min_flow:
                    coupling = GradientDrivenCoupling(
                        conductivity=flow / 100.0,
                        state_coupling=0.1
                    )
                    net.add_coupling(f'cell_{i}', f'cell_{j}', coupling)
                    n_edges += 1
    
    return net, top_indices

network, selected_cells = create_sweep_network(amazon_data, SWEEP_CONFIG)
print(f"Created network: {network.n_elements} nodes, {network.number_of_edges()} edges")

## 5. Worker Function

In [ ]:
def run_single_forcing_experiment(args):
    """
    Worker function for a single forcing experiment.
    """
    network_bytes, recovery_forcing, config, seed = args
    
    import sys
    import numpy as np
    import pickle
    
    if '/opt/research-local/src' not in sys.path:
        sys.path.insert(0, '/opt/research-local/src')
    
    from energy_constrained.solvers import run_two_phase_experiment
    
    network = pickle.loads(network_bytes)
    np.random.seed(seed)
    
    # Run with forcing
    result = run_two_phase_experiment(
        network=network,
        cascade_duration=config['cascade_duration'],
        recovery_duration=config['recovery_duration'],
        dt=config['dt'],
        cascade_sigma=config['cascade_sigma'],
        cascade_alpha=config['cascade_alpha'],
        recovery_sigma=config['recovery_sigma'],
        recovery_alpha=config['recovery_alpha'],
        recovery_forcing=recovery_forcing,  # THE SWEEP VARIABLE
        seed=seed
    )
    
    # Compute additional metrics
    pct_time_tipped = np.mean(result.x_full > 0) * 100
    
    n_cells = result.x_full.shape[1]
    n_tip_events = 0
    n_recover_events = 0
    
    for j in range(n_cells):
        x_traj = result.x_full[:, j]
        signs = np.sign(x_traj)
        sign_changes = np.diff(signs)
        n_tip_events += np.sum(sign_changes > 0)
        n_recover_events += np.sum(sign_changes < 0)
    
    return {
        'recovery_forcing': recovery_forcing,
        'seed': seed,
        'recovery_fraction': result.metrics['recovery_fraction'],
        'pct_tipped_cascade': result.metrics['pct_tipped_at_cascade_end'],
        'n_permanent_tips': result.metrics['n_permanent_tips'],
        'final_pct_tipped': result.metrics['final_pct_tipped'],
        'pct_time_tipped': pct_time_tipped,
        'n_tip_events': n_tip_events,
        'n_recover_events': n_recover_events,
        'mean_recovery_time': result.metrics['mean_recovery_time'],
    }

print("Worker function defined.")

## 6. Run Forcing Sweep

In [ ]:
# Serialize and scatter network
network_bytes = pickle.dumps(network)
print(f"Network serialized: {len(network_bytes) / 1024:.1f} KB")

network_future = client.scatter(network_bytes, broadcast=True)
print("Network broadcast to all workers")

# Build task arguments
print("\n" + "=" * 60)
print("EXPERIMENT 10c: Starting Forcing Sweep")
print("=" * 60)
start_time = time.time()

task_args = []
for i, forcing in enumerate(SWEEP_CONFIG['forcing_values']):
    for run_idx in range(SWEEP_CONFIG['n_runs_per_forcing']):
        seed = SWEEP_CONFIG['base_seed'] + i * 1000 + run_idx
        task_args.append((network_bytes, float(forcing), SWEEP_CONFIG, seed))

print(f"Generated {len(task_args)} task arguments")

# Submit using client.map
futures = client.map(run_single_forcing_experiment, task_args)
print(f"Submitted {len(futures)} tasks")

# Collect results
all_results = []
print("\nProgress:")

for i, future in enumerate(as_completed(futures)):
    result = future.result()
    all_results.append(result)
    
    if (i + 1) % 20 == 0:
        elapsed = time.time() - start_time
        print(f"  Completed {i+1}/{len(futures)} ({100*(i+1)/len(futures):.1f}%)")

elapsed = time.time() - start_time
print(f"\n" + "=" * 60)
print(f"COMPLETE: {len(all_results)} simulations in {elapsed:.1f}s ({elapsed/60:.1f} min)")
print("=" * 60)

## 7. Results Aggregation

In [ ]:
df = pd.DataFrame(all_results)
print(f"Results shape: {df.shape}")

df = df.sort_values(['recovery_forcing', 'seed']).reset_index(drop=True)
df.head(10)

In [ ]:
# Summary statistics by forcing level
summary = df.groupby('recovery_forcing').agg({
    'recovery_fraction': ['mean', 'std', 'min', 'max'],
    'n_permanent_tips': ['mean', 'std'],
    'final_pct_tipped': ['mean', 'std'],
    'mean_recovery_time': ['mean', 'std'],
    'n_recover_events': ['mean', 'std'],
}).round(4)

summary.columns = ['_'.join(col) for col in summary.columns]
summary = summary.reset_index()

print("=" * 80)
print("FORCING SWEEP SUMMARY")
print("=" * 80)
print(summary[['recovery_forcing', 'recovery_fraction_mean', 'recovery_fraction_std',
               'n_permanent_tips_mean', 'mean_recovery_time_mean']].to_string(index=False))

## 8. Primary Visualization

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

forcing_vals = summary['recovery_forcing'].values

# Panel 1: Recovery Fraction vs Forcing (KEY RESULT)
ax = axes[0, 0]
means = summary['recovery_fraction_mean'].values
stds = summary['recovery_fraction_std'].values
ax.errorbar(-forcing_vals, means, yerr=stds, marker='o', capsize=4, 
            linewidth=2, markersize=10, color='green')
ax.axhline(0.5, color='orange', linestyle='--', alpha=0.7, linewidth=2, label='50% recovery')
ax.axhline(0.1, color='red', linestyle=':', alpha=0.7, linewidth=2, label='10% recovery')
ax.set_xlabel('Restoration Forcing Strength (|f|)', fontsize=12)
ax.set_ylabel('Recovery Fraction', fontsize=12)
ax.set_title('Recovery vs Restoration Forcing', fontsize=14)
ax.set_ylim(0, 1.05)
ax.legend()
ax.grid(True, alpha=0.3)

# Panel 2: Permanent Tips vs Forcing
ax = axes[0, 1]
means = summary['n_permanent_tips_mean'].values
stds = summary['n_permanent_tips_std'].values
ax.errorbar(-forcing_vals, means, yerr=stds, marker='s', capsize=4,
            linewidth=2, markersize=10, color='red')
ax.set_xlabel('Restoration Forcing Strength (|f|)', fontsize=12)
ax.set_ylabel('Permanent Tips (out of 50)', fontsize=12)
ax.set_title('Irreversibility vs Restoration Forcing', fontsize=14)
ax.grid(True, alpha=0.3)

# Panel 3: Recovery Time vs Forcing
ax = axes[1, 0]
means = summary['mean_recovery_time_mean'].values
stds = summary['mean_recovery_time_std'].values
# Filter out NaN values for plotting
valid_idx = ~np.isnan(means)
if np.any(valid_idx):
    ax.errorbar(-forcing_vals[valid_idx], means[valid_idx], yerr=stds[valid_idx],
                marker='^', capsize=4, linewidth=2, markersize=10, color='blue')
ax.set_xlabel('Restoration Forcing Strength (|f|)', fontsize=12)
ax.set_ylabel('Mean Recovery Time', fontsize=12)
ax.set_title('Recovery Speed vs Restoration Forcing', fontsize=14)
ax.grid(True, alpha=0.3)

# Panel 4: Box plot of recovery fraction by forcing
ax = axes[1, 1]
data_by_forcing = [df[df['recovery_forcing'] == f]['recovery_fraction'].values 
                   for f in SWEEP_CONFIG['forcing_values']]
bp = ax.boxplot(data_by_forcing, 
                labels=[f'{-f:.2f}' for f in SWEEP_CONFIG['forcing_values']], 
                patch_artist=True)

# Color boxes by recovery level
colors = plt.cm.RdYlGn(np.linspace(0, 1, len(SWEEP_CONFIG['forcing_values'])))
for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)

ax.axhline(0.5, color='orange', linestyle='--', alpha=0.7)
ax.set_xlabel('Restoration Forcing Strength (|f|)', fontsize=12)
ax.set_ylabel('Recovery Fraction', fontsize=12)
ax.set_title('Recovery Distribution by Forcing Level', fontsize=14)
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('/workspace/data/exp10c_forcing_results.png', dpi=150, bbox_inches='tight')
plt.show()
print("\nPlot saved to /workspace/data/exp10c_forcing_results.png")

## 9. Critical Forcing Analysis

In [ ]:
def find_critical_forcing(summary_df, recovery_threshold=0.5):
    """
    Find minimum forcing needed to achieve recovery_threshold.
    """
    # Sort by forcing (most negative first)
    subset = summary_df.sort_values('recovery_forcing')
    forcings = subset['recovery_forcing'].values
    recoveries = subset['recovery_fraction_mean'].values
    
    # Find crossing point (recovery increasing as forcing becomes more negative)
    for i in range(len(forcings) - 1):
        if recoveries[i] >= recovery_threshold > recoveries[i+1]:
            # Linear interpolation
            slope = (recoveries[i+1] - recoveries[i]) / (forcings[i+1] - forcings[i])
            if slope != 0:
                critical = forcings[i] + (recovery_threshold - recoveries[i]) / slope
                return critical
    
    # Check if always above or below
    if recoveries.max() < recovery_threshold:
        return None  # Never reaches threshold
    elif recoveries.min() >= recovery_threshold:
        return forcings.max()  # Even weakest forcing is enough
    
    return None


print("=" * 60)
print("CRITICAL FORCING ANALYSIS")
print("=" * 60)

for thresh in [0.10, 0.30, 0.50, 0.70, 0.90]:
    critical = find_critical_forcing(summary, thresh)
    if critical is not None:
        print(f"\n{thresh*100:.0f}% recovery requires forcing: f = {critical:.3f} (|f| = {-critical:.3f})")
    else:
        print(f"\n{thresh*100:.0f}% recovery: threshold not reached in tested range")

## 10. Physical Interpretation

In [ ]:
print("=" * 70)
print("PHYSICAL INTERPRETATION")
print("=" * 70)

# Compare forcing to barrier height
barrier = SWEEP_CONFIG['barrier_height']
print(f"\nBarrier height: {barrier}")
print(f"\nForcing as fraction of barrier:")

for forcing in SWEEP_CONFIG['forcing_values']:
    if forcing != 0:
        ratio = -forcing / barrier
        recovery = summary[summary['recovery_forcing'] == forcing]['recovery_fraction_mean'].values[0]
        print(f"  f = {forcing:.2f}: |f|/barrier = {ratio:.1%}, recovery = {recovery:.1%}")

# Estimate real-world forcing
print("\n" + "=" * 70)
print("REAL-WORLD IMPLICATIONS")
print("=" * 70)
print("""
The forcing term f represents the intensity of active restoration efforts:

  |f| = 0.1: Moderate intervention
    - Protected areas with fire suppression
    - Sustainable forestry practices
    - Local reforestation projects

  |f| = 0.3: Strong intervention  
    - Large-scale reforestation programs
    - Moisture corridor restoration
    - Regional conservation networks

  |f| = 0.5: Maximum intervention
    - Full ecosystem restoration
    - Complete deforestation ban
    - Massive investment in recovery

The critical forcing threshold tells us the MINIMUM intervention
intensity needed to achieve meaningful recovery.
""")

## 11. Key Findings Summary

In [ ]:
print("\n" + "=" * 70)
print("EXPERIMENT 10c: KEY FINDINGS")
print("=" * 70)

# Find critical forcings
critical_50 = find_critical_forcing(summary, 0.50)
critical_10 = find_critical_forcing(summary, 0.10)

# Best and worst cases
best = summary.loc[summary['recovery_fraction_mean'].idxmax()]
worst = summary.loc[summary['recovery_fraction_mean'].idxmin()]

# Format critical thresholds safely
crit_10_str = f"{-critical_10:.3f}" if critical_10 is not None else "N/A (not reached)"
crit_50_str = f"{-critical_50:.3f}" if critical_50 is not None else "N/A (not reached)"

print(f"""
1. PASSIVE RECOVERY (f=0):
   Recovery fraction: {summary[summary['recovery_forcing']==0]['recovery_fraction_mean'].values[0]:.3f}
   Confirms Experiments 10/10b: passive recovery is impossible

2. CRITICAL FORCING THRESHOLDS:
   10% recovery requires: |f| >= {crit_10_str}
   50% recovery requires: |f| >= {crit_50_str}

3. BEST RECOVERY ACHIEVED:
   Forcing: f = {best['recovery_forcing']:.2f}
   Recovery fraction: {best['recovery_fraction_mean']:.3f}
   Permanent tips: {best['n_permanent_tips_mean']:.1f}

4. FORCING-RECOVERY RELATIONSHIP:
""")

# Analyze linearity
forcings = summary['recovery_forcing'].values
recoveries = summary['recovery_fraction_mean'].values
# Only fit non-zero forcings
mask = forcings != 0
if np.sum(mask) > 2:
    slope, intercept = np.polyfit(-forcings[mask], recoveries[mask], 1)
    print(f"   Linear fit: recovery ≈ {slope:.2f} * |f| + {intercept:.3f}")
    print(f"   Each 0.1 increase in |f| adds ~{slope*0.1:.1%} recovery")

## 12. Save Results

In [ ]:
df.to_csv('/workspace/data/experiment10c_forcing_results.csv', index=False)
print(f"Full results saved to /workspace/data/experiment10c_forcing_results.csv")

summary.to_csv('/workspace/data/experiment10c_summary.csv', index=False)
print(f"Summary saved to /workspace/data/experiment10c_summary.csv")

In [ ]:
print("\n" + "=" * 70)
print("EXPERIMENT 10c COMPLETE")
print("=" * 70)

print(f"""
CONFIGURATION:
- Forcing values: {list(SWEEP_CONFIG['forcing_values'])}
- Runs per level: {SWEEP_CONFIG['n_runs_per_forcing']}
- Total simulations: {len(df)}
- Runtime: {elapsed:.1f}s ({elapsed/60:.1f} min)

FILES GENERATED:
- /workspace/data/experiment10c_forcing_results.csv
- /workspace/data/experiment10c_summary.csv  
- /workspace/data/exp10c_forcing_results.png

KEY CONCLUSION:
Recovery from tipping requires active intervention (forcing).
The critical forcing threshold quantifies the minimum restoration
effort needed to reverse ecosystem collapse.
""")